# Goal

The goal is to segment customers based on their purchasing behavior so the company can better target marketing strategies and identify high-value customer groups. Business stakeholders could use these clusters to design loyalty programs, identify high-value vs low-value customers, and optimize promotions.

# Loading the Data

In [1]:
import pandas as pd

In [5]:
df = pd.read_excel('../data/raw/online_retail_II.xlsx')

## Dataset Description
- `InvoiceNo`: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation. 
- `StockCode`: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product. 
- `Description`: Product (item) name. Nominal. 
- `Quantity`: The quantities of each product (item) per transaction. Numeric.	
- `InvoiceDate`: Invice date and time. Numeric. The day and time when a transaction was generated. 
- `UnitPrice`: Unit price. Numeric. Product price per unit in sterling (Â£). 
- `CustomerID`: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer. 
- `Country`: Country name. Nominal. The name of the country where a customer resides.

In [6]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [7]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 122.9 MB


The dataset is huge. We don't really need the `Description` in this dataframe, so it can be split into a separate table to save memory. The `Country` could be converted to categorical. `CustomerID` has many null values, making these rows useless to us since we want to understand specific customer behaviour. We should also convert it to int.

# Data Validation

## Customer ID

In [8]:
df = df.dropna(subset='Customer ID')

In [9]:
df['Customer ID'] = df['Customer ID'].astype(int)

In [10]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 417534 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      417534 non-null  object        
 1   StockCode    417534 non-null  object        
 2   Description  417534 non-null  object        
 3   Quantity     417534 non-null  int64         
 4   InvoiceDate  417534 non-null  datetime64[ns]
 5   Price        417534 non-null  float64       
 6   Customer ID  417534 non-null  int64         
 7   Country      417534 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 100.8 MB


There aren't any null values anymore.

## Invoice

In [11]:
df[df['Invoice'].astype(str).str.contains('C')]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321,Australia
...,...,...,...,...,...,...,...,...
524695,C538123,22956,36 FOIL HEART CAKE CASES,-2,2010-12-09 15:41:00,2.10,12605,Germany
524696,C538124,M,Manual,-4,2010-12-09 15:43:00,0.50,15329,United Kingdom
524697,C538124,22699,ROSES REGENCY TEACUP AND SAUCER,-1,2010-12-09 15:43:00,2.95,15329,United Kingdom
524698,C538124,22423,REGENCY CAKESTAND 3 TIER,-1,2010-12-09 15:43:00,12.75,15329,United Kingdom


Are cancelled invoices recorded as reversals of previously completed transactions, meaning I need to subtract them from the original values, or are they the only record of the transaction, allowing me to simply exclude them from the analysis as if the purchase never occurred?

In [12]:
df[df['Customer ID'] == 16321]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321,Australia
183,C489449,21871,SAVE THE PLANET MUG,-12,2009-12-01 10:33:00,1.25,16321,Australia
184,C489449,84946,ANTIQUE SILVER TEA GLASS ETCHED,-12,2009-12-01 10:33:00,1.25,16321,Australia
185,C489449,84970S,HANGING HEART ZINC T-LIGHT HOLDER,-24,2009-12-01 10:33:00,0.85,16321,Australia
186,C489449,22090,PAPER BUNTING RETRO SPOTS,-12,2009-12-01 10:33:00,2.95,16321,Australia
187,489450,22087,PAPER BUNTING WHITE LACE,12,2009-12-01 10:36:00,2.95,16321,Australia


This customer cancelled an order three minutes before placing the exact same one. This likely indicates a checkout error, such as incorrect payment details or a delivery issue, which led them to cancel and re-order. This behavior suggests that cancelled transactions were not previously recorded as completed orders. Therefore, they should be treated as incomplete and can be safely excluded from customer value analysis.

In [13]:
df = df[~df['Invoice'].astype(str).str.startswith('C')]
df['Invoice'] = df['Invoice'].astype(int)

## StockCode

In [14]:
df['StockCode'].nunique()

4017

In [15]:
df[~df['StockCode'].astype(str).str.isdigit()]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
12,489436,48173C,DOOR MAT BLACK FLOCK,10,2009-12-01 09:06:00,5.95,13078,United Kingdom
23,489436,35004B,SET OF 3 BLACK FLYING DUCKS,12,2009-12-01 09:06:00,4.65,13078,United Kingdom
28,489436,84596F,SMALL MARSHMALLOWS PINK BOWL,8,2009-12-01 09:06:00,1.25,13078,United Kingdom
...,...,...,...,...,...,...,...,...
525387,538170,84029E,RED WOOLLY HOTTIE WHITE HEART.,2,2010-12-09 19:32:00,3.75,13969,United Kingdom
525388,538170,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,2,2010-12-09 19:32:00,3.75,13969,United Kingdom
525389,538170,85232B,SET OF 3 BABUSHKA STACKING TINS,2,2010-12-09 19:32:00,4.95,13969,United Kingdom
525435,538171,47591D,PINK FAIRY CAKE CHILDRENS APRON,1,2010-12-09 20:01:00,1.95,17530,United Kingdom


Looks like the characters at the end of stock codes indicates a variant of the same item, but with different price. So we can't combine them into one code and remove the characters.

In [21]:
product_code = df['StockCode'].astype(str).str.extract(r'(\d+)\w?').iloc[:, 0]
product_code

0         85048
1         79323
2         79323
3         22041
4         21232
          ...  
525456    22271
525457    22750
525458    22751
525459    20970
525460    21931
Name: 0, Length: 407695, dtype: object

In [25]:
df[product_code.astype(str).str.len() != 5]['StockCode'].unique()

array(['POST', 'C2', 'M', 'BANK CHARGES', 'TEST001', 'TEST002', 'PADS',
       'ADJUST', 'D', 'ADJUST2', 'SP1002'], dtype=object)

In [26]:
df = df[product_code.astype(str).str.len() == 5]
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530,United Kingdom


As stated by the dataset description, the stock code should be a 5 digit code for each product. However, it sometimes included characters at the end for variants of the same product. When filtering on codes that had 5 digits at the beginning, we seem to have removed the unwanted stock codes and retained only the item purchases.

## Quantity

In [27]:
df['Quantity'].describe()

count    406323.000000
mean         13.619534
std          97.002302
min           1.000000
25%           2.000000
50%           5.000000
75%          12.000000
max       19152.000000
Name: Quantity, dtype: float64

In [28]:
large_quantities_df = df[df['Quantity'] > df['Quantity'].quantile(0.99)]
large_quantities_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
217,489460,84598,BOYS ALPHABET IRON ON PATCHES,576,2009-12-01 10:46:00,0.21,16167,United Kingdom
223,489460,84568,GIRLS ALPHABET IRON ON PATCHES,288,2009-12-01 10:46:00,0.21,16167,United Kingdom
282,489465,84879,ASSORTED COLOUR BIRD ORNAMENT,160,2009-12-01 10:52:00,1.45,13767,United Kingdom
575,489523,84879,ASSORTED COLOUR BIRD ORNAMENT,800,2009-12-01 11:46:00,1.45,12931,United Kingdom
934,489539,22086,PAPER CHAIN KIT 50'S CHRISTMAS,160,2009-12-01 12:18:00,2.55,15061,United Kingdom
...,...,...,...,...,...,...,...,...
522809,537899,22328,ROUND SNACK BOXES SET OF 4 FRUITS,1488,2010-12-09 10:44:00,2.55,12755,Japan
522917,537981,22560,TRADITIONAL MODELLING CLAY,250,2010-12-09 11:35:00,1.06,17857,United Kingdom
522918,537981,22492,MINI PAINT SET VINTAGE,1394,2010-12-09 11:35:00,0.55,17857,United Kingdom
522919,537981,22536,MAGIC DRAWING SLATE PURDEY,400,2010-12-09 11:35:00,0.36,17857,United Kingdom


The highest 1% of quantity orders appear legitimate, as they are associated with valid customer IDs and stock codes. Given the company’s business model, selling unique all-occasion gift-ware mostly to wholesalers, such high-volume purchases are expected. These orders likely originate from wholesale customers or bulk buyers. Since they reflect genuine business activity, they will be retained in the dataset and are expected to form their own cluster during customer segmentation.

## InvoiceDate

In [29]:
print(f'First Invoice: {df['InvoiceDate'].min()}')
print(f'Last Invoice: {df['InvoiceDate'].max()}')

First Invoice: 2009-12-01 07:45:00
Last Invoice: 2010-12-09 20:01:00


In [30]:
df.sort_values('InvoiceDate').head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom


In [31]:
df.sort_values('InvoiceDate').tail()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
525423,538171,20771,CHRYSANTHEMUM JOURNAL,1,2010-12-09 20:01:00,2.55,17530,United Kingdom
525424,538171,21882,SKULLS TAPE,5,2010-12-09 20:01:00,0.65,17530,United Kingdom
525425,538171,21879,HEARTS GIFT TAPE,5,2010-12-09 20:01:00,0.65,17530,United Kingdom
525410,538171,22417,PACK OF 60 SPACEBOY CAKE CASES,3,2010-12-09 20:01:00,0.55,17530,United Kingdom
525460,538171,21931,JUMBO STORAGE BAG SUKI,2,2010-12-09 20:01:00,1.95,17530,United Kingdom


Everything seems fine.

## Price

In [32]:
df['Price'].describe()

count    406323.000000
mean          2.991565
std           4.285914
min           0.000000
25%           1.250000
50%           1.950000
75%           3.750000
max         295.000000
Name: Price, dtype: float64

In [33]:
df[df['Price'] == 0].drop_duplicates('StockCode')

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
4674,489825,22076,6 RIBBONS EMPIRE,12,2009-12-02 13:34:00,0.0,16126,United Kingdom
6781,489998,48185,DOOR MAT FAIRY CAKE,2,2009-12-03 11:19:00,0.0,15658,United Kingdom
18738,490961,22065,CHRISTMAS PUDDING TRINKET POT,1,2009-12-08 15:25:00,0.0,14108,United Kingdom
18739,490961,22142,CHRISTMAS CRAFT WHITE FAIRY,12,2009-12-08 15:25:00,0.0,14108,United Kingdom
32916,492079,85042,ANTIQUE LILY FAIRY LIGHTS,8,2009-12-15 13:49:00,0.0,15070,United Kingdom
40101,492760,21143,ANTIQUE GLASS HEART DECORATION,12,2009-12-18 14:22:00,0.0,18071,United Kingdom
47126,493761,79320,FLAMINGO LIGHTS,24,2010-01-06 14:54:00,0.0,14258,United Kingdom
48342,493899,22355,"CHARLOTTE BAG , SUKI DESIGN",10,2010-01-08 10:43:00,0.0,12417,Belgium
57619,494607,21533,RETRO SPOT LARGE MILK JUG,12,2010-01-15 12:43:00,0.0,16858,United Kingdom
111348,500073,21662,VINTAGE GLASS COFFEE CADDY,1,2010-03-04 11:44:00,0.0,13047,United Kingdom


In [34]:
df[df['Price'] > df['Price'].quantile(0.99)].drop_duplicates('StockCode').sort_values('Price', ascending=False)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
268526,515349,22655,VINTAGE RED KITCHEN CABINET,1,2010-07-12 10:43:00,295.00,15513,United Kingdom
268525,515349,22656,VINTAGE BLUE KITCHEN CABINET,1,2010-07-12 10:43:00,295.00,15513,United Kingdom
406342,528325,22827,GIANT SEVENTEEN DRAWER SIDEBOARD,1,2010-10-21 13:02:00,165.00,15680,United Kingdom
406996,528363,22828,REGENCY MIRROR WITH SHUTTERS,2,2010-10-21 15:38:00,145.00,14442,Channel Islands
106631,499601,21760,FRENCH STYLE WALL DRESSER,1,2010-03-01 12:34:00,125.00,13018,United Kingdom
...,...,...,...,...,...,...,...,...
391414,527048,22838,3 TIER CAKE TIN RED AND CREAM,1,2010-10-14 12:49:00,14.95,15827,United Kingdom
390754,526933,22849,"BREAD BIN, DINER STYLE, MINT",4,2010-10-14 09:44:00,14.95,13696,United Kingdom
382945,526214,22762,CUPBOARD 3 DRAWER MA CAMPAGNE,2,2010-10-10 14:17:00,14.95,12749,United Kingdom
7034,490012,21303,FILIGREE WALL MIRROR ANTIQUE WHITE,4,2009-12-03 12:27:00,14.75,15179,United Kingdom


Both extremes seem valid. The free orders could be coupons, promos or samples. The expensive products are reasonable.

## Country

In [35]:
df['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'EIRE', 'Germany',
       'Portugal', 'Denmark', 'Netherlands', 'Poland', 'Channel Islands',
       'Spain', 'Cyprus', 'Belgium', 'Greece', 'Norway', 'Austria',
       'Sweden', 'United Arab Emirates', 'Finland', 'Italy',
       'Switzerland', 'USA', 'Japan', 'Unspecified', 'Malta', 'RSA',
       'Singapore', 'Bahrain', 'Thailand', 'Israel', 'Lithuania',
       'Nigeria', 'West Indies', 'Korea', 'Brazil', 'Canada', 'Iceland'],
      dtype=object)

In [36]:
df['Country'].value_counts()

Country
United Kingdom          370474
EIRE                      8388
Germany                   7383
France                    5288
Netherlands               2662
Spain                     1195
Switzerland               1146
Belgium                    989
Portugal                   956
Sweden                     841
Channel Islands            817
Italy                      695
Australia                  625
Cyprus                     539
Greece                     511
Austria                    504
Denmark                    408
Norway                     362
Finland                    338
United Arab Emirates       313
Unspecified                277
USA                        225
Poland                     181
Malta                      168
Japan                      164
Lithuania                  154
Singapore                  117
Canada                      77
Thailand                    76
Israel                      74
Iceland                     71
RSA                         65


In [37]:
df['Country'] = df['Country'].astype('category')

No issues with the countries. Only converted to categorical to save memory.

## Splitting the data

In [38]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 406323 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      406323 non-null  int64         
 1   StockCode    406323 non-null  object        
 2   Description  406323 non-null  object        
 3   Quantity     406323 non-null  int64         
 4   InvoiceDate  406323 non-null  datetime64[ns]
 5   Price        406323 non-null  float64       
 6   Customer ID  406323 non-null  int64         
 7   Country      406323 non-null  category      
dtypes: category(1), datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 63.3 MB


In [39]:
product_df = df[['StockCode', 'Description']].drop_duplicates('StockCode')
order_df = df.drop('Description', axis=1)

In [40]:
order_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 406323 entries, 0 to 525460
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      406323 non-null  int64         
 1   StockCode    406323 non-null  object        
 2   Quantity     406323 non-null  int64         
 3   InvoiceDate  406323 non-null  datetime64[ns]
 4   Price        406323 non-null  float64       
 5   Customer ID  406323 non-null  int64         
 6   Country      406323 non-null  category      
dtypes: category(1), datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 33.9 MB


In [41]:
product_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 4006 entries, 0 to 523069
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   StockCode    4006 non-null   object
 1   Description  4006 non-null   object
dtypes: object(2)
memory usage: 494.3 KB


# Acquiring Customer Data

In [42]:
customer_df = order_df.groupby('Customer ID').agg({
    'Invoice': 'nunique',
    'Quantity': 'sum',
    'Price': 'sum',
    'InvoiceDate': ['min', 'max'],
    'StockCode': pd.Series.nunique,
    'Country': 'first'
})

customer_df.columns = [
    'NumOrders', 'TotalQuantity', 'TotalSpend',
    'FirstPurchaseDate', 'LastPurchaseDate',
    'UniqueProducts', 'Country'
]

In [43]:
customer_df.head()

,NumOrders,TotalQuantity,TotalSpend,FirstPurchaseDate,LastPurchaseDate,UniqueProducts,Country
Customer ID,,,,,,,
12346,2,24,169.36,2010-03-02 13:08:00,2010-06-28 13:53:00,24,United Kingdom
12347,2,828,162.95,2010-10-31 14:20:00,2010-12-07 14:57:00,70,Iceland
12348,1,372,13.39,2010-09-27 14:59:00,2010-09-27 14:59:00,19,Finland
12349,2,991,425.34,2010-04-29 13:20:00,2010-10-28 08:23:00,89,Italy
12351,1,261,49.46,2010-11-29 15:23:00,2010-11-29 15:23:00,21,Unspecified


In [44]:
order_df.to_csv('../data/clean/orders.csv', index=False)
product_df.to_csv('../data/clean/products.csv', index=False)
customer_df.to_csv('../data/clean/customers.csv')